# Goals

* Check on the test and/or production databases

In [22]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [23]:
import os
import pandas as pd
from pypika import Query, Table, Field, Column, Criterion

In [24]:
from SRAgent.db.connect import db_connect
from SRAgent.db.upsert import db_upsert
from SRAgent.db.utils import db_list_tables, db_glimpse_tables, db_get_table, execute_query
from SRAgent.db.get import db_find_srx
from SRAgent.db.create import create_table, create_table_router

In [25]:
# list all of the tables in prod
os.environ['DYNACONF'] = 'prod'
conn_prod = db_connect() 
print("\n".join(db_list_tables(conn_prod)))

screcounter_star_results
eval
screcounter_trace
srx_srr
srx_metadata
screcounter_log
screcounter_star_params


# Production

In [26]:
db_list_tables(conn_prod)

['screcounter_star_results',
 'eval',
 'screcounter_trace',
 'srx_srr',
 'srx_metadata',
 'screcounter_log',
 'screcounter_star_params']

In [27]:
db_glimpse_tables(conn_prod)

#-- Table: screcounter_star_results --#
sample	feature	estimated_number_of_cells	fraction_of_unique_reads_in_cells	mean_gene_per_cell	mean_umi_per_cell	mean_feature_per_cell	median_gene_per_cell	median_umi_per_cell	median_feature_per_cell	number_of_reads	median_reads_per_cell	q30_bases_in_cb_umi	q30_bases_in_rna_read	reads_mapped_to_gene__unique_gene	reads_mapped_to_gene__unique_multiple_gene	reads_mapped_to_genefull__unique_genefull	reads_mapped_to_genefull__unique_multiple_genefull	reads_mapped_to_genefull_ex50pas__unique_genefull_ex50pas	reads_mapped_to_genefull_ex50pas__unique_multiple_genefull_ex50	reads_mapped_to_genefull_exonoverintron__unique_genefull_exonov	reads_mapped_to_genefull_exonoverintron__unique_multiple_genefu	reads_mapped_to_genome__unique	reads_mapped_to_genome__unique_multiple	reads_mapped_to_velocyto__unique_velocyto	reads_mapped_to_velocyto__unique_multiple_velocyto	reads_with_valid_barcodes	sequencing_saturation	total_feature_detected	umis_in_cells	unique_reads

# Database updates

## CZI datasets

### Organism 

In [28]:
# read in srx-metadata as pandas dataframe
tbl = Table("srx_metadata")
stmt = Query \
    .from_(tbl) \
    .select(tbl.star) \
    .distinct() \
    .where(tbl.czi_collection_id.isnull().negate())
srx_metadata_cxg = pd.read_sql(str(stmt), conn_prod)
srx_metadata_cxg

,database,entrez_id,srx_accession,is_illumina,is_single_cell,is_paired_end,lib_prep,tech_10x,cell_prep,organism,tissue,disease,purturbation,cell_line,czi_collection_id,czi_collection_name,notes,created_at,updated_at
0,sra,5980061,SRX4401543,yes,no,no,other,not_applicable,not_applicable,human,Human prostate basal epithelia,unsure,unsure,"D4PrF_BE, CD326+/CD271+/CD26-",e2a4a67f-6a18-431a-ab9c-6e77dd31cc80,A Cellular Anatomy of the Normal Adult Human P...,Metadata obtained by SRAgent,2025-01-03 18:39:40.221883,2025-02-18 01:17:18.338743
1,sra,19008386,SRX13671170,yes,yes,yes,other,not_applicable,single_nucleus,human,Dorsolateral Prefrontal cortex (Brodmann area 46),unsure,unsure,unsure,91c8e321-566f-4f9d-b89e-3a164be654d5,Neuron type-specific effects of human aging an...,Metadata obtained by SRAgent,2025-01-03 18:39:40.221883,2025-02-18 01:48:15.592726
2,sra,21270522,ERX8792046,yes,yes,no,10x_Genomics,5_prime_gex,single_cell,human,Lung-draining lymph node,Prostate cancer,Saquinavir-NO treatment,PC-3,62ef75e4-cbea-454e-a0ce-998ec40223d3,Cross-tissue immune cell analysis reveals tiss...,Metadata obtained by SRAgent,2025-01-03 18:39:40.221883,2025-02-16 03:08:04.360701
3,sra,33748102,ERX12098959,yes,yes,yes,10x_Genomics,3_prime_gex,single_cell,human,intercostal muscle,intracranial haemorrhage,unsure,nuclei from human skeletal muscle biopsies,2d40e6a7-f2fd-49ba-9db9-6b97e4c6dad5,Human skeletal muscle ageing atlas,Metadata obtained by SRAgent,2025-01-03 18:39:40.221883,2025-02-15 15:20:42.842344
4,sra,30534579,SRX22540304,yes,yes,yes,10x_Genomics,5_prime_gex,single_cell,human,Peripheral Blood Mononuclear Cells (PBMCs),COVID-19,COVID-19 mRNA vaccine (BNT162b2),PBMCs,ecb739c5-fe0d-4b48-81c6-217c4d64eec4,COVID-19 mRNA vaccine elicits a potent adaptiv...,Metadata obtained by SRAgent,2025-01-03 18:39:40.221883,2025-02-15 23:28:22.014208
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6055,sra,30533995,SRX22539720,yes,yes,yes,10x_Genomics,5_prime_gex,single_cell,human,PBMCs,COVID-19,COVID-19 vaccination,PBMCs,ecb739c5-fe0d-4b48-81c6-217c4d64eec4,COVID-19 mRNA vaccine elicits a potent adaptiv...,Metadata obtained by SRAgent,2025-01-03 18:39:40.221883,2025-02-17 01:40:39.811736
6056,sra,19008361,SRX13671145,yes,no,yes,other,not_applicable,single_nucleus,human,Dorsolateral Prefrontal Cortex (Brodmann area 46),Alzheimer's Disease,unsure,unsure,91c8e321-566f-4f9d-b89e-3a164be654d5,Neuron type-specific effects of human aging an...,Metadata obtained by SRAgent,2025-01-03 18:39:40.221883,2025-02-17 03:59:44.582294
6057,sra,5987222,SRX4405260,yes,yes,no,MARS-seq,not_applicable,single_cell,human,Bone Marrow,Multiple myeloma (MM),Untreated,Control donor (AB2835),2a0b02c0-fea6-47bd-92b9-9b03f5d2580c,Single cell dissection of plasma cell heteroge...,Metadata obtained by SRAgent,2025-01-03 18:39:40.221883,2025-02-15 14:20:46.277606
6058,sra,9278658,SRX7059587,yes,yes,yes,10x_Genomics,3_prime_gex,single_cell,human,brain,"glioblastoma, stage IV",unsure,unsure,999f2a15-3d7e-440b-96ae-2c806799c08c,"Harmonized single-cell landscape, intercellula...",Metadata obtained by SRAgent,2025-01-03 18:39:40.221883,2025-02-15 03:37:30.793431


In [29]:
# summarize organism
srx_metadata_cxg.groupby("organism").size().sort_values(ascending=False)

organism
human                4990
mouse                1005
Macaca mulatta         36
NaN                    14
Rattus norvegicus       5
Sus scrofa              4
Danio rerio             3
Gallus gallus           2
other                   1
dtype: int64

In [30]:
# filter to just `other` organisms
srx_metadata_cxg[srx_metadata_cxg["organism"] == "other"][["srx_accession", "organism"]]

,srx_accession,organism
3641,SRX13549222,other


In [12]:
# # update "other" organisms to correct
# idx = [
#     ['SRX9556667', 'human'], 
#     ['ERX12060278', 'human'], 
#     ['ERX12060274', 'human'], 
#     ['ERX12060276', 'human'], 
#     ['ERX12060273', 'human'], 
#     ['ERX12060297', 'human'], 
#     ['ERX12060299', 'human'], 
#     ['ERX12060275', 'human'], 
#     ['SRX13549222', 'other'],
#     ['SRX9556651', 'human'], 
#     ['ERX12060277', 'human'], 
# ]
# idx = pd.DataFrame(idx, columns=["srx_accession", "organism"])
# idx

,srx_accession,organism
0,SRX9556667,human
1,ERX12060278,human
2,ERX12060274,human
3,ERX12060276,human
4,ERX12060273,human
5,ERX12060297,human
6,ERX12060299,human
7,ERX12060275,human
8,SRX13549222,other
9,SRX9556651,human


In [17]:
# with with other columns
srx_metadata_cxg_f = pd.merge(srx_metadata_cxg.drop(columns=["organism"]), idx, on="srx_accession", how="inner")
srx_metadata_cxg_f = srx_metadata_cxg_f[["database", "entrez_id", "srx_accession", "organism"]]
srx_metadata_cxg_f

,database,entrez_id,srx_accession,organism
0,sra,12488082,SRX9556667,human
1,sra,32114110,ERX12060278,human
2,sra,32114006,ERX12060274,human
3,sra,32113144,ERX12060276,human
4,sra,32114450,ERX12060273,human
5,sra,32114882,ERX12060297,human
6,sra,32113802,ERX12060299,human
7,sra,32113700,ERX12060275,human
8,sra,18806356,SRX13549222,other
9,sra,12488066,SRX9556651,human


In [19]:
# update records
from SRAgent.db.connect import db_connect
from SRAgent.db.update import db_update

In [20]:
with db_connect() as conn:
    db_update(srx_metadata_cxg_f, "srx_metadata", conn)

# --OLD--

### Filter out records with no SRX accessions

`New dataset found by Find-Datasets agent`

In [8]:
entrez_ids = srx_metadata[srx_metadata["srx_accession"].isna()]["entrez_id"].unique().tolist()
entrez_ids

[]

In [13]:
# delete extrez_ids from srx_metadata
tbl = Table("srx_metadata")
stmt = Query \
    .from_(tbl) \
    .delete() \
    .where(tbl.entrez_id.isin(entrez_ids))
with conn_prod.cursor() as cur:
    cur.execute(str(stmt))
    conn_prod.commit()

In [16]:
# delete from log
tbl = Table("screcounter_log")
to_rm = ["ERX11146221"]
stmt = Query \
    .from_(tbl) \
    .delete() \
    .where(tbl.sample.isin(to_rm))
with conn_prod.cursor() as cur:
    cur.execute(str(stmt))
    conn_prod.commit()